# 변경 X

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import time

# 넷플릭스 한국 데이터 가져오기

In [2]:
nf_df = pd.read_csv('../data/korea_netflix.csv')

In [3]:
korea_netflix_df = nf_df.copy()
korea_netflix_df = korea_netflix_df.rename(columns={'Unnamed: 0':'original index'})
# korea_netflix_df = korea_netflix_df.drop(df.columns[0], axis=1)
korea_netflix_df.head()

,original index,Title,Release Date,Series or Movies,drama,comedy,action,thriller,romance,crime,...,music,short,sport,war,musical,reality-tv,western,game-show,talk-show,Title for Crawling
0,4,Moxie,2011-09-22,Movie,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,moxie
1,27,Comrades Almost a Love Story,1996-11-02,Movie,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,comradesalmostalovestory
2,28,The Closet,2001-08-10,Movie,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,thecloset
3,53,The Bold Type,2017-06-20,Series,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,theboldtype
4,55,Lobster Cop,2018-06-22,Movie,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,lobstercop


- title_for_crawling_list 로 크롤링

In [ ]:
title_for_crawling_list = korea_netflix_df[korea_netflix_df['Series or Movies']=='Movie']['Title for Crawling'].values.tolist()
title_for_crawling_list

In [4]:
BASE_URL = "https://movie.naver.com"
SEARCH = "/movie/search/result.naver?query="
POINT_SEARCH = "/movie/bi/mi/point.naver?code="

In [5]:
movie_df = pd.DataFrame(columns=['code', 'korean_title', 'english_title', 'genre_list', 'country', 'released_date', 'director', 'actors'])

In [6]:
def html_parser(url):
    req = requests.get(url)
    return req.text

In [7]:
def html_selector(html, query):
    soup = BeautifulSoup(html, 'html.parser')
    return soup.select(query)

In [8]:
def get_point_url(code):
    return html_parser(BASE_URL + POINT_SEARCH + str(code))

# 영화 정보 크롤링

code, 한국 제목, 장르, 국가, 개봉일(null값있음), director, actors(의미없는 값 있음)

개봉일과 actors는 1.5만 데이터에서도 확인할 수 있으므로 더 이상 디테일은 잡지 않았습니다.

ENG_TITLE_LIST에 영화 제목들을 담아주시면 되는데, 띄어쓰기는 모두 replace 또는 strip으로 없애주세요!

대소문자는 상관없습니다.

In [9]:
def get_movie_info(search_title):
#     print(search_title)
    try:
        search_html = html_parser(BASE_URL + SEARCH + search_title)
        search_result = html_selector(search_html, '.search_list_1 > li')
        # 가장 첫 번째로 나온 결과만을 가져감
        result = str(search_result[0].select('dl > dt > a'))
    except:
        return 'error'

    movie_url = result.split("href=")[1].split(">")[0].replace('"', '')
    code = movie_url.split('=')[1]

    basic_html = html_parser(BASE_URL + movie_url)

    search_result = html_selector(basic_html, 'h3.h_movie > a')
    title = str(search_result[0]).split('>')[1].split('<')[0]

    movie_result = html_selector(basic_html, 'dl.info_spec > dd')
    temp_summary = movie_result[0].select('a')
    director = movie_result[1].text
    actors = movie_result[2].text.split(',')
    
    genre_list = list()
    nation_list = list()
    released = list()
    for summary in temp_summary:
        c = summary.attrs['href'].split(".naver?")[1].split('=')[0]

        if c == 'genre':
            genre_list.append(summary.text)
        elif c == 'nation':
            nation_list.append(summary.text)
        elif c == 'open':
            released.append(summary.text)
        else:
            print(c)
    
    return {'code': code, 'korean_title': title, 'english_title': search_title, 'genre_list': genre_list, 'country': nation_list, 'released_date': ''.join(released), 'director': director, 'actors': actors}

In [ ]:
target_index = title_for_crawling_list.index('operationchristmasdrop')
print(target_index)
korea_netflix_df[korea_netflix_df['Series or Movies']=='Movie'].iloc[[target_index], :]

- error at if loop에서 걸린 영화 : 넷플릭스 영화가 대부분
- error at except에서 걸린 영화 : 네이버 무비에 등록은 되어 있는데 국내 개봉 안됨 등의 이유로 평점정보가 (일부) 없는 경우

- 예외: thethreelittlepigs 같은 경우 The Three Little Pigs and KungFu Land 로 검색하면 나오긴 하는데 if loop에서 걸림.

In [ ]:
# 예시
# ENG_TITLE_LIST = ['kingsman', 'spiderman', 'HOWTOBUILDAGIRL', 'Centigrade', 'LetsFightGhost', 'Moxie']


# ENG_TITLE_LIST = title_for_crawling_list

In [ ]:
'''
print로 나온 친구들은 크롤링에 실패한 아이들입니다. 무슨 이유인지 간혹 있더라구요.
목록 다시 넘겨주시면 이유를 제가 다시 한 번 확인해보겠습니다!
'''

error_at_if = list()
error_at_except = list()
for i, eng_title in enumerate(ENG_TITLE_LIST):

    
    if i % 50 == 0:
        print(f'@@@@@@@@@@@@@{i+1}번째 작업 중@@@@@@@@@@@@')
    try:
        result = get_movie_info(eng_title)
        if result == 'error':
            print('error at if loop: ',eng_title)
            error_at_if.append(eng_title)
        else:
            movie_df = movie_df.append(result, ignore_index=True)
    except:
        error_at_except.append(eng_title)
        print('error at exept: ', eng_title)
        
print(f'{len(ENG_TITLE_LIST)}개 작업')
print(f'성공: {len(ENG_TITLE_LIST) - len(error_at_if) - len(error_at_except)} 개')
print(f'if에서 {len(error_at_if)}개, except에서 {len(error_at_except)}개, 총 {len(error_at_if) + len(error_at_except)} 실패')


In [10]:
movie_df

,code,korean_title,english_title,genre_list,country,released_date,director,actors


####  크롤링 데이터 저장 - get_movie_info()
- 3075개 작업
- 성공: 1854 개
- if에서 880개, except에서 341개, 총 1221 실패

In [ ]:
# movie_df.to_csv('../data/crawling_movie_info.csv', sep=',', encoding='utf-8-sig')

In [ ]:
# with open("../data/failed_crawling/crawling_movie_info_failed_if.txt", "w") as output:
#     output.write(str(error_at_if))

In [ ]:
# with open("../data/failed_crawling/crawling_movie_info_failed_except.txt", "w") as output:
#     output.write(str(error_at_except))

# 네티즌 평점 크롤링

movie_df에서 가져온 code를 기준으로 다시 평점 데이터를 받아옴

아직 미개봉 영화에 대해서는 모두 NaN으로 처리

`code`: 영화코드(str)

`num_of_netizen`: 평가에 참여한 네티즌의 수(str)

`key_review`: 한 줄 리뷰(str)

`score_male`: 남자 평점(str)

`score_female`: 여자 평점(str)

`scores_per_age`: 나이별 평점(float list) (10대, 20대, 30대, 40대, 50대 이상 순)

######  get_movie_info에서 크롤링한 내용 가져오기

In [11]:
movie_df = pd.read_csv('../data/crawling_movie_info_modified.csv')
movie_df

,code,korean_title,english_title,genre_list,country,released_date,director,actors
0,203096,걸스 오브 막시,moxie,['드라마'],['미국'],NaN,에이미 포엘러,"['조세핀 랭포드', ' 패트릭 슈왈제네거더보기']"
1,17997,첨밀밀,comradesalmostalovestory,"['멜로/로맨스', '드라마']",['홍콩'],1997.03.01,진가신,"['여명(소군)', ' 장만옥(이요)더보기']"
2,181925,클로젯,thecloset,"['미스터리', '드라마']",['한국'],2020.02.05,김광빈,"['하정우(상원)', ' 김남길(경훈)', ' 허율(이나)더보기']"
3,195979,비밀경찰: 랍스터 캅,lobstercop,"['액션', '코미디']",['중국'],NaN,이흔운,"['왕첸웬', ' 원산산', ' 류화더보기']"
4,138113,매직 아워,magichour,['범죄'],['미국'],NaN,콜 뮐러,"['트레버 모건(레이)', ' 크리스 렘세(딜런 폭스)', ' 윌리엄 러스(브루스 ..."
...,...,...,...,...,...,...,...,...
1819,72043,쓰리 데이즈,thenextthreedays,"['범죄', '드라마', '멜로/로맨스', '스릴러']",['미국'],2010.12.22,폴 해기스,"['러셀 크로우(존 브레넌)', ' 엘리자베스 뱅크스(라라 브레넌)더보기']"
1820,77566,브레이킹 던 part2,thetwilightsagabreakingdawnpart2,"['드라마', '판타지', '멜로/로맨스', '스릴러']",['미국'],2012.11.15,빌 콘돈,"['크리스틴 스튜어트(벨라 컬렌)', ' 로버트 패틴슨(에드워드 컬렌)', ' 테일..."
1821,38227,첫 키스만 50번째,50firstdates,"['멜로/로맨스', '코미디']",['미국'],2017.06.22 2004.04.15,피터 시걸,"['아담 샌들러(헨리 로스)', ' 드류 베리모어(루시 윗모어)더보기']"
1822,74567,브레이킹 던 part1,thetwilightsagabreakingdawnpart1,"['드라마', '판타지', '멜로/로맨스', '스릴러']",['미국'],2011.11.30,빌 콘돈,"['크리스틴 스튜어트(벨라 스완)', ' 로버트 패틴슨(에드워드 컬렌)더보기']"


In [12]:
def get_review_info(code, classification):
    
    idx = 1 if classification == 'audience' else 0
    point_url = get_point_url(code)
    
    try:
        elements = html_selector(point_url, f'.grade_{classification} > .title_area.grade_tit > .sc_area > .star_score > .st_off > .st_on')
        score = elements[0].attrs['style'].split(':')[1]
    except:
        return 'error'

    num = html_selector(point_url, f'.grade_{classification} > .title_area.grade_tit > .sc_area > .user_count > em')[0].text
    
    
    # key_review가 없는 경우 index out of range 에러가 뜨길래 수정했습니다.
    try:
        key_review = html_selector(point_url, '.grp_review')[0].text
    except:
        key_review = None
        
    male = html_selector(point_url, '.grp_gender > .grp_box > .grp_male > .graph_point')[idx].text
    female = html_selector(point_url, '.grp_gender > .grp_box > .grp_female > .graph_point')[idx].text
    score_list = [float(element.text) for element in html_selector(point_url, '.grp_age > .grp_box > .graph_point')[idx*5:(idx+1)*5]]
    
    result = {'code': code, f'num_of_{classification}': num, 'key_review': key_review, 'score_male': male, 'score_female': female, 'scores_per_age': score_list}
    
    if classification == 'audience':
        num_per_ages = [element.attrs['style'].split(': ')[1] for element in html_selector(point_url, '.bar_graph > .graph_box > .graph > span')[:5]]
        result['num_per_ages'] = num_per_ages

    return result

In [13]:
netizen_df = movie_df[['code', 'korean_title']].copy()
netizen_df

,code,korean_title
0,203096,걸스 오브 막시
1,17997,첨밀밀
2,181925,클로젯
3,195979,비밀경찰: 랍스터 캅
4,138113,매직 아워
...,...,...
1819,72043,쓰리 데이즈
1820,77566,브레이킹 던 part2
1821,38227,첫 키스만 50번째
1822,74567,브레이킹 던 part1


In [ ]:
start_netizen_len = len(netizen_df['code'])
print(start_netizen_len)

In [ ]:
# ntz_count = 0

In [ ]:
# error_at_if_ntz = list()
# error_at_except_ntz = list()

# def start_netizen(x):
#     global ntz_count
#     if ntz_count % 50 == 0:
#         print(f'@@@@@@@@@@@@@{ntz_count+1}번째 작업 중@@@@@@@@@@@@')
#     try:
#         if get_review_info(x, 'netizen') != 'error':
#             get_review_info(x, 'netizen').tolist()
#             ntz_count += 1
#         else :
#             print(f'error at if loop: {x}')
#             error_at_if_ntz.append(x)
#             x = 'error'
#             ntz_count += 1
            
#     except:
#         print(f'error at except: {x}')
#         error_at_except_ntz.append(x)
#         ntz_count += 1

# print(f'{start_netizen_len}개 작업')
# print(f'성공: {start_netizen_len - len(error_at_if_ntz) - len(error_at_except_ntz)} 개')
# print(f'if에서 {len(error_at_if_ntz)}개, except에서 {len(error_at_except_ntz)}개, 총 {len(error_at_if_ntz) + len(error_at_except_ntz)} 실패')

- error at if loop: 리뷰 수가 극히 적어서 일부 데이터 없음
- error at except: 국내 개봉 안한 영화라서 리뷰 없음

- 이전에 실행할 때 netizen_df를 리셋 안하고 함수를 중복 실행해서 중복된 rows들이 500여개 가량 있었는데

   drop_duplicates로 모두 날려버렸습니다.
   
   현재 1824개

In [ ]:
data = netizen_df['code'].apply(lambda x: get_review_info(x, 'netizen') if get_review_info(x, 'netizen') != 'error' else 'error').tolist()
data = list(filter(lambda x: x != 'error', data))
temp_df = pd.DataFrame(data, columns=['code', 'num_of_netizen', 'key_review', 'score_male', 'score_female', 'scores_per_age'])
netizen_test_df = pd.merge(netizen_df, temp_df, how='outer', on='code')
netizen_test_df

- netizen_df가 현재 중복row 삭제된 완전체입니다. 

In [14]:
netizen_df.head()

,code,korean_title
0,203096,걸스 오브 막시
1,17997,첨밀밀
2,181925,클로젯
3,195979,비밀경찰: 랍스터 캅
4,138113,매직 아워


In [15]:
netizen_pp_df.head()

NameError: name 'netizen_pp_df' is not defined

In [16]:
netizen_pp_df = netizen_test_df.drop_duplicates(subset=['code'])

NameError: name 'netizen_test_df' is not defined

In [ ]:
netizen_pp_df = netizen_pp_df.reset_index(drop=True)

In [ ]:
netizen_pp_df

####  csv 파일로 저장

In [ ]:
# netizen_pp_df.to_csv('../data/crawling_netizen_review.csv', sep=',', encoding='utf-8-sig')

In [17]:
netizen_saved_df = pd.read_csv('../data/crawling_netizen_review.csv')
netizen_saved_df

,code,korean_title,num_of_netizen,key_review,score_male,score_female,scores_per_age
0,203096,걸스 오브 막시,140,이 영화는 20대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,6.12,9.86,"[9.76, 9.83, 8.92, 9.38, 10.0]"
1,17997,첨밀밀,"1,851",이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,9.19,9.13,"[8.86, 9.05, 9.27, 9.48, 9.35]"
2,181925,클로젯,"6,242",이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,5.55,7.04,"[7.66, 6.11, 5.95, 6.11, 6.33]"
3,195979,비밀경찰: 랍스터 캅,NaN,NaN,NaN,NaN,NaN
4,138113,매직 아워,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1819,72043,쓰리 데이즈,"1,627",이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,8.39,8.03,"[7.65, 8.41, 8.51, 8.34, 8.26]"
1820,77566,브레이킹 던 part2,"7,053",이 영화는 10대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,7.95,9.20,"[9.22, 8.59, 7.95, 8.76, 8.08]"
1821,38227,첫 키스만 50번째,"3,682",이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,9.26,9.24,"[9.41, 9.17, 9.15, 9.22, 8.98]"
1822,74567,브레이킹 던 part1,"3,290",이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,5.41,7.97,"[7.71, 6.25, 5.33, 6.46, 9.1]"


In [18]:
netizen_saved_df.isna().sum()

code                0
korean_title        0
num_of_netizen    617
key_review        650
score_male        617
score_female      617
scores_per_age    617
dtype: int64

In [19]:
netizen_saved_modified_df = netizen_saved_df[~netizen_saved_df['num_of_netizen'].isna()]
netizen_saved_modified_df

,code,korean_title,num_of_netizen,key_review,score_male,score_female,scores_per_age
0,203096,걸스 오브 막시,140,이 영화는 20대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,6.12,9.86,"[9.76, 9.83, 8.92, 9.38, 10.0]"
1,17997,첨밀밀,"1,851",이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,9.19,9.13,"[8.86, 9.05, 9.27, 9.48, 9.35]"
2,181925,클로젯,"6,242",이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,5.55,7.04,"[7.66, 6.11, 5.95, 6.11, 6.33]"
5,197517,마리오네트,121,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,7.42,8.25,"[10.0, 7.89, 8.28, 7.52, 6.6]"
6,155356,미쓰백,"8,671",이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,8.29,9.60,"[9.69, 9.36, 8.7, 8.55, 8.51]"
...,...,...,...,...,...,...,...
1819,72043,쓰리 데이즈,"1,627",이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,8.39,8.03,"[7.65, 8.41, 8.51, 8.34, 8.26]"
1820,77566,브레이킹 던 part2,"7,053",이 영화는 10대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,7.95,9.20,"[9.22, 8.59, 7.95, 8.76, 8.08]"
1821,38227,첫 키스만 50번째,"3,682",이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,9.26,9.24,"[9.41, 9.17, 9.15, 9.22, 8.98]"
1822,74567,브레이킹 던 part1,"3,290",이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,5.41,7.97,"[7.71, 6.25, 5.33, 6.46, 9.1]"


- 수정된 df csv 파일로 저장

In [ ]:
# netizen_saved_modified_df.to_csv('../data/crawling_netizen_review_modified.csv', sep=',', encoding='utf-8-sig')

In [20]:
load_netizen_saved_modified_df = pd.read_csv('../data/crawling_netizen_review_modified.csv')
load_netizen_saved_modified_df
                                             

,code,korean_title,num_of_netizen,key_review,score_male,score_female,scores_per_age
0,203096,걸스 오브 막시,140,이 영화는 20대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,6.12,9.86,"[9.76, 9.83, 8.92, 9.38, 10.0]"
1,17997,첨밀밀,"1,851",이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,9.19,9.13,"[8.86, 9.05, 9.27, 9.48, 9.35]"
2,181925,클로젯,"6,242",이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,5.55,7.04,"[7.66, 6.11, 5.95, 6.11, 6.33]"
3,197517,마리오네트,121,이 영화는 40대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,7.42,8.25,"[10.0, 7.89, 8.28, 7.52, 6.6]"
4,155356,미쓰백,"8,671",이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,8.29,9.60,"[9.69, 9.36, 8.7, 8.55, 8.51]"
...,...,...,...,...,...,...,...
1202,72043,쓰리 데이즈,"1,627",이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,8.39,8.03,"[7.65, 8.41, 8.51, 8.34, 8.26]"
1203,77566,브레이킹 던 part2,"7,053",이 영화는 10대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,7.95,9.20,"[9.22, 8.59, 7.95, 8.76, 8.08]"
1204,38227,첫 키스만 50번째,"3,682",이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,9.26,9.24,"[9.41, 9.17, 9.15, 9.22, 8.98]"
1205,74567,브레이킹 던 part1,"3,290",이 영화는 10대 여성이 좋아하는 연기가 뛰어난 영화입니다.,5.41,7.97,"[7.71, 6.25, 5.33, 6.46, 9.1]"


# 관람객 평점

네티즌 평점과 동일하고, 추가사항이 있음

`num_per_ages`: 나이대 별 관람객 수 비율(10대, 20대, 30대, 40대, 50대 이상 순)

`num_per_genders`: 성별 별 관람객 수 비율 (남자, 여자 순)

In [21]:
audience_df = movie_df[['code', 'korean_title']].copy()
audience_df

,code,korean_title
0,203096,걸스 오브 막시
1,17997,첨밀밀
2,181925,클로젯
3,195979,비밀경찰: 랍스터 캅
4,138113,매직 아워
...,...,...
1819,72043,쓰리 데이즈
1820,77566,브레이킹 던 part2
1821,38227,첫 키스만 50번째
1822,74567,브레이킹 던 part1


In [22]:
data = audience_df['code'].apply(lambda x: get_review_info(x, 'audience') if get_review_info(x, 'audience') != 'error' else 'error').tolist()
data = list(filter(lambda x: x != 'error', data))
temp_df = pd.DataFrame(data, columns=['code', 'num_of_audience', 'key_review', 'score_male', 'score_female', 'scores_per_age', 'num_per_ages'])
audience_df = pd.merge(audience_df, temp_df, how='outer', on='code')
audience_df

,code,korean_title,num_of_audience,key_review,score_male,score_female,scores_per_age,num_per_ages
0,203096,걸스 오브 막시,NaN,NaN,NaN,NaN,NaN,NaN
1,17997,첨밀밀,NaN,NaN,NaN,NaN,NaN,NaN
2,181925,클로젯,387,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,7.93,8.41,"[8.75, 7.75, 8.17, 8.79, 8.63]","[2%, 44%, 30%, 15%, 9%]"
3,195979,비밀경찰: 랍스터 캅,NaN,NaN,NaN,NaN,NaN,NaN
4,138113,매직 아워,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1819,72043,쓰리 데이즈,NaN,NaN,NaN,NaN,NaN,NaN
1820,77566,브레이킹 던 part2,NaN,NaN,NaN,NaN,NaN,NaN
1821,38227,첫 키스만 50번째,177,이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,9.35,9.08,"[9.78, 9.08, 9.7, 9.0, 9.12]","[5%, 68%, 15%, 7%, 5%]"
1822,74567,브레이킹 던 part1,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
code_list = audience_df['code'].tolist()

- 예상시간 90분 소요
- 1916i

In [25]:
audience_genders = {}

driver = webdriver.Chrome('../code/soryeongk/chromedriver.exe')
for code in code_list:
    driver.get(BASE_URL + POINT_SEARCH + str(code))
    time.sleep(3)
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    temp_genders = bs.find_all('tspan')
    if len(temp_genders) <= 5:
        num_per_genders = np.nan
    else:
        num_per_genders = [element.text for element in temp_genders[:2]]
    audience_genders[code] = num_per_genders

<ipython-input-25-6efadda32c25>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../code/soryeongk/chromedriver.exe')


UnexpectedAlertPresentException: Alert Text: 영화 코드값 오류입니다.
Message: unexpected alert open: {Alert text : 영화 코드값 오류입니다.}
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x00276903+2517251]
	Ordinal0 [0x0020F8E1+2095329]
	Ordinal0 [0x00112848+1058888]
	Ordinal0 [0x001659FB+1399291]
	Ordinal0 [0x0015639B+1336219]
	Ordinal0 [0x001327A7+1189799]
	Ordinal0 [0x00133609+1193481]
	GetHandleVerifier [0x00405904+1577972]
	GetHandleVerifier [0x004B0B97+2279047]
	GetHandleVerifier [0x00306D09+534521]
	GetHandleVerifier [0x00305DB9+530601]
	Ordinal0 [0x00214FF9+2117625]
	Ordinal0 [0x002198A8+2136232]
	Ordinal0 [0x002199E2+2136546]
	Ordinal0 [0x00223541+2176321]
	BaseThreadInitThunk [0x7546FA29+25]
	RtlGetAppContainerNamedObjectPath [0x76F17A9E+286]
	RtlGetAppContainerNamedObjectPath [0x76F17A6E+238]
	(No symbol) [0x00000000]


In [26]:
audience_genders

{203096: nan,
 17997: nan,
 181925: ['58%', '42%'],
 195979: nan,
 138113: nan,
 197517: ['100%', '21%'],
 155356: ['36%', '64%'],
 168668: ['44%', '56%'],
 156023: nan,
 151155: nan,
 143776: ['38%', '62%'],
 168735: ['22%', '78%'],
 142683: ['25%', '75%'],
 199742: nan,
 167675: ['100%', '24%'],
 151241: ['61%', '39%'],
 154667: ['36%', '64%'],
 164269: ['38%', '62%'],
 159366: ['26%', '74%'],
 162874: ['28%', '72%'],
 158180: ['45%', '55%'],
 167638: ['42%', '58%'],
 180209: ['37%', '63%'],
 161834: ['50%', '50%'],
 172454: ['54%', '46%'],
 158112: ['43%', '57%'],
 167651: ['45%', '55%'],
 164192: ['55%', '45%'],
 171744: nan,
 143422: ['25%', '75%'],
 130811: ['100%', '9%'],
 195980: nan,
 168749: ['41%', '59%'],
 142210: ['33%', '67%'],
 159892: ['44%', '56%'],
 164150: ['100%', '19%'],
 158191: ['50%', '50%'],
 194581: nan,
 142255: ['44%', '56%'],
 199262: nan,
 194416: nan,
 190313: nan,
 179338: ['100%', '30%'],
 190244: ['100%', '19%'],
 193100: nan,
 190341: nan,
 177332: ['

In [27]:
audience_df = audience_df.iloc[:, :-1]

In [28]:
temp_df = pd.DataFrame(columns=['code', 'num_per_genders'])
temp_df['code'] = audience_genders.keys()
temp_df['num_per_genders'] = audience_genders.values()

audience_df = pd.merge(audience_df, temp_df, how='outer', on='code')

C:\Users\Las\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [29]:
audience_df

,code,korean_title,num_of_audience,key_review,score_male,score_female,scores_per_age,num_per_genders
0,203096,걸스 오브 막시,NaN,NaN,NaN,NaN,NaN,NaN
1,17997,첨밀밀,NaN,NaN,NaN,NaN,NaN,NaN
2,181925,클로젯,387,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,7.93,8.41,"[8.75, 7.75, 8.17, 8.79, 8.63]","[58%, 42%]"
3,195979,비밀경찰: 랍스터 캅,NaN,NaN,NaN,NaN,NaN,NaN
4,138113,매직 아워,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1819,72043,쓰리 데이즈,NaN,NaN,NaN,NaN,NaN,NaN
1820,77566,브레이킹 던 part2,NaN,NaN,NaN,NaN,NaN,NaN
1821,38227,첫 키스만 50번째,177,이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,9.35,9.08,"[9.78, 9.08, 9.7, 9.0, 9.12]",NaN
1822,74567,브레이킹 던 part1,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
audience_df.isna().sum()

code                  0
korean_title          0
num_of_audience    1230
key_review         1230
score_male         1230
score_female       1230
scores_per_age     1230
num_per_genders    1398
dtype: int64

- csv로 저장

In [31]:
# audience_df.to_csv('../data/crawling_audience_review.csv', sep=',', encoding='utf-8-sig')

In [32]:
audience_review_loaded_df = pd.read_csv('../data/crawling_audience_review.csv')
audience_review_loaded_df

,Unnamed: 0,code,korean_title,num_of_audience,key_review,score_male,score_female,scores_per_age,num_per_genders
0,0,203096,걸스 오브 막시,NaN,NaN,NaN,NaN,NaN,NaN
1,1,17997,첨밀밀,NaN,NaN,NaN,NaN,NaN,NaN
2,2,181925,클로젯,387,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,7.93,8.41,"[8.75, 7.75, 8.17, 8.79, 8.63]","['58%', '42%']"
3,3,195979,비밀경찰: 랍스터 캅,NaN,NaN,NaN,NaN,NaN,NaN
4,4,138113,매직 아워,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1819,1819,72043,쓰리 데이즈,NaN,NaN,NaN,NaN,NaN,NaN
1820,1820,77566,브레이킹 던 part2,NaN,NaN,NaN,NaN,NaN,NaN
1821,1821,38227,첫 키스만 50번째,177,이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,9.35,9.08,"[9.78, 9.08, 9.7, 9.0, 9.12]",NaN
1822,1822,74567,브레이킹 던 part1,NaN,NaN,NaN,NaN,NaN,NaN


# 연습 공간

In [ ]:
point_url = get_point_url(CODE)
netizen_elements = html_selector(point_url, '.grade_audience > .title_area.grade_tit > .sc_area > .star_score > .st_off > .st_on')
score = netizen_elements[0].attrs['style'].split(':')[1]
num_netizen = html_selector(point_url, '.grade_audience > .title_area.grade_tit > .sc_area > .user_count > em')[0].text

In [ ]:
netizen_male = html_selector(point_url, '.grp_gender > .grp_box > .grp_male > .graph_point')[1].text
netizen_female = html_selector(point_url, '.grp_gender > .grp_box > .grp_female > .graph_point')[1].text

In [ ]:
score_list = [float(element.text) for element in html_selector(point_url, '.grp_age > .grp_box > .graph_point')[5:]]

In [ ]:
def get_netizen_info(code):
    try:
        point_url = get_point_url(code)
        netizen_elements = html_selector(point_url, '.grade_netizen > .title_area.grade_tit > .sc_area > .star_score > .st_off > .st_on')
        score = netizen_elements[0].attrs['style'].split(':')[1]
    except:
        return 'error'

    num_netizen = html_selector(point_url, '.grade_netizen > .title_area.grade_tit > .sc_area > .user_count > em')[0].text
    key_review = html_selector(point_url, '.grp_review')[0].text
    netizen_male = html_selector(point_url, '.grp_gender > .grp_box > .grp_male > .graph_point')[0].text
    netizen_female = html_selector(point_url, '.grp_gender > .grp_box > .grp_female > .graph_point')[0].text
    score_list = [float(element.text) for element in html_selector(point_url, '.grp_age > .grp_box > .graph_point')[:5]]
    
    return {'code': code, 'num_of_netizen': num_netizen, 'key_review': key_review, 'score_male': netizen_male, 'score_female': netizen_female, 'scores_per_age': score_list}

In [ ]:
point_url = html_parser(BASE_URL + POINT_SEARCH + str(31307))
netizen_elements = html_selector(point_url, '.grade_netizen > .title_area.grade_tit > .sc_area > .star_score > .st_off > .st_on')
# netizen_score = html_selector(netizen_elements, '.star_score > .st_off > .st_on')
score = netizen_elements[0].attrs['style'].split(':')[1]
num_netizen = html_selector(point_url, '.grade_netizen > .title_area.grade_tit > .sc_area > .user_count > em')[0].text

In [ ]:
key_review = html_selector(point_url, '.grp_review')[0].text

In [ ]:
netizen_male = html_selector(point_url, '.grp_gender > .grp_box > .grp_male > .graph_point')[0].text
netizen_female = html_selector(point_url, '.grp_gender > .grp_box > .grp_female > .graph_point')[0].text

In [ ]:
score_list = [float(element.text) for element in html_selector(point_url, '.grp_age > .grp_box > .graph_point')]
score_list

In [ ]:
# example: google search
# driver.get("http://www.google.com")
# input_element = driver.find_element_by_name("q")
# input_element.send_keys("python")
# input_element.submit()

In [ ]:
search_title = 'kingsman'
search_html = html_parser(BASE_URL + SEARCH + search_title)
search_result = html_selector(search_html, '.search_list_1 > li')
result = str(search_result[0].select('dl > dt > a'))
movie_url = result.split("href=")[1].split(">")[0].replace('"', '')
code = movie_url.split('=')[1]

basic_html = html_parser(BASE_URL + movie_url)

search_result = html_selector(basic_html, 'h3.h_movie > a')
title = str(search_result[0]).split('>')[1].split('<')[0]

movie_result = html_selector(basic_html, 'dl.info_spec > dd')
temp_summary = movie_result[0].select('a')
director = movie_result[1].text
actors = movie_result[2].text.split(',')

In [ ]:
print(code, title, temp_summary, director, actors)

In [ ]:
genre = list()
nation = list()
released = list()

In [ ]:
for summary in temp_summary:
    c = summary.attrs['href'].split(".naver?")[1].split('=')[0]
    if c == 'genre':
        genre.append(summary.text)
    elif c == 'nation':
        nation.append(summary.text)
    elif c == 'open':
        released.append(summary.text)
    else:
        print(c)

In [ ]:
genre, nation, released

In [ ]:
''.join(released)

In [ ]:
movie_df

In [ ]:
BASE_URL = "https://movie.naver.com/movie/search/result.naver?query=스파이더맨"
req = requests.get(BASE_URL)
html = req.text
soup = BeautifulSoup(html, 'html.parser')
# CSS Selector를 통해 html요소들을 찾아낸다.
search_result_list = soup.select(
    '.search_list_1 > li'
    )
temp = str(search_result_list[0].select('dl > dt > a'))

In [ ]:
temp.split("href=")[1].split(">")[0].replace('"', '')

In [ ]:
def search_naver_movie(title):
    req = requests.get(BASE_URL + SEARCH + title)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    search_result_list = soup.select('.search_list_1 > li')
    result = str(search_result_list[0].select('dl > dt > a'))
    movie_url = result.split("href=")[1].split(">")[0].replace('"', '')
    
    req = requests.get(BASE_URL + movie_url)
    html = req.text

In [ ]:
# movie_data_set = {'code': 0, 'title': '', 'genre_list': [], 'country': [], 'runtime': '', 'released_date': '', 'director': '', 'actors': []}

In [ ]:
movie_df # -> korean title, english title

In [ ]:
netizen_df

In [ ]:
audience_df # -> 영민님이 말씀하신 부분 추가

In [ ]:
temp_summary[0]

In [ ]:
list(map(lambda x: x.text, list(temp_summary)))

In [ ]:
BASE_URL = "https://movie.naver.com/movie/search/result.naver?query=트랜스포머"
req = requests.get(BASE_URL)
html = req.text
soup = BeautifulSoup(html, 'html.parser')
# CSS Selector를 통해 html요소들을 찾아낸다.
my_titles = soup.select(
    '.search_list_1 > li'
    )
my_titles[0].select('dl > dt > a')

In [ ]:
BASE_URL = "https://movie.naver.com/movie/search/result.naver?query=시월애"
req = requests.get(BASE_URL)
html = req.text
soup = BeautifulSoup(html, 'html.parser')
# CSS Selector를 통해 html요소들을 찾아낸다.
my_titles = soup.select(
    '.search_list_1 > li'
    )
my_titles[0].select('dl > dt > a')

In [ ]:
driver = webdriver.Chrome(executable_path='./chromedriver')
driver.get(BASE_URL)
input_element = driver.find_element_by_class_name("search_list_1")
print(input_element.text)

In [ ]:
def search_naver_movie(title):
    BASE_URL = "https://movie.naver.com/movie/search/result.naver?query=스파이더맨"
    req = requests.get(BASE_URL)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    # CSS Selector를 통해 html요소들을 찾아낸다.
    search_result_list = soup.select(
        '.search_list_1 > li'
        )
    search_result_list[0]